In [1]:
import osmium
import sys
from neo4j import GraphDatabase, basic_auth, unit_of_work
from copy import deepcopy

In [2]:
url = 'http://download.geofabrik.de/europe/monaco-latest.osm.pbf'

In [3]:
class GraphNode:

    def __init__(self, node_id, lat, long):
        self.node_id = node_id
        self.lat = lat
        self.long = long

    def __repr__(self):
        return f'{{node_id: {self.node_id}, lat: {self.lat}, long: {self.long}}}'

In [4]:
class GraphEdge:

    def __init__(self, start_node_id, end_node_id, distance, rating):
        self.start_node_id = start_node_id
        self.end_node_id = end_node_id
        self.distance = distance
        self.rating = rating

In [5]:
class Neo4jHandler():
    
    def __init__(self):
        self.driver = GraphDatabase.driver(
            "bolt://54.167.190.221:7687",
            auth=basic_auth("neo4j", "release-shows-breads"))
        self.nodes = []
        self.edges = []
        
    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, edge):
        self.edges.append(edge)

    def create_objects(self):
        with self.driver.session() as session:
            batch = []
            for i, node in enumerate(self.nodes):
                if i % 2500 == 0 and len(batch) > 0:
                    with session.begin_transaction() as tx:
                        tx.run(f'''FOREACH (node IN [{','.join(map(lambda x: str(x), batch))}]|
                               CREATE ({{node_id:node.node_id,lat:node.lat,long:node.long}}))''')
                        tx.commit()
                        print(f'Close transaction {i//2500}')
                        batch = []

#                 print(i)
                batch.append(node)
    
    def close(self):
        self.driver.close()

In [6]:
class FileHandler(osmium.SimpleHandler):

    def __init__(self, neoHandler):
        super(FileHandler, self).__init__()
        self.neoHandler = neoHandler

    def node(self, n):
        id = deepcopy(n.id)
        lat, lon = deepcopy(str(n.location)).split('/')
        node = GraphNode(n.id, lat, lon)
        self.neoHandler.add_node(node)

#     def way(self, w):
#         print(w)

#     def relation(self, r):
#         print(r)


In [7]:
%%time
neoHandler = Neo4jHandler()
h = FileHandler(neoHandler)

h.apply_file(url, locations=True)

neoHandler.create_objects()
neoHandler.close()

NameError: name 'FileStatsHandler' is not defined